# Project of ana and leo

In [1]:
import pandas as pd
import statistics
import warnings
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from function import Utils
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from pandas.api.types import is_numeric_dtype
import numpy as np
import h2o
from IPython.display import display
warnings.filterwarnings('ignore')

## Data loading

In [2]:
df = pd.read_csv("hmw/Sampled_BPIC17_offer_log.csv")

In [5]:
df = df[0:int(len(df)/2)]

## Data preprocessing

In [6]:
input = {
    "Case ID" : "caseid",
    "Activity" : "activity" ,
    "Resource" : "resource" ,
    "Complete Timestamp": "ts",
    "(case) Accepted" : "y"  #Needs to be 1 for each Case
}

# Change columns name
df.rename(columns=input,inplace = True)

# Timestamp value in df
df["ts"] = pd.to_datetime(df["ts"])

df["dt"] = Utils.dt(df) # Thisneed to be improved
df.drop("ts",axis=1,inplace = True)

In [7]:
Utils.persantage_nan(df)

index                            0.000000
caseid                           0.000000
activity                         0.000000
resource                         0.000000
Variant                          0.000000
Variant index                    0.000000
y                                0.000000
(case) ApplicationID             0.000000
(case) CreditScore               0.000000
(case) FirstWithdrawalAmount     0.000000
(case) MonthlyCost               0.000000
(case) NumberOfTerms             0.000000
(case) OfferedAmount             0.000000
(case) Selected                  0.000000
Action                           0.000000
EventID                          0.000000
EventOrigin                      0.000000
OfferID                         22.313459
lifecycle:transition             0.000000
dt                               0.000000
dtype: float64

In [8]:
df = Utils.predict_null_value(colum="OfferID",df=df)

In [9]:
df = Utils.prod_nan_with_treshold(df)
df.head()

,index,caseid,activity,resource,Variant,Variant index,y,(case) ApplicationID,(case) CreditScore,(case) FirstWithdrawalAmount,...,(case) NumberOfTerms,(case) OfferedAmount,(case) Selected,Action,EventID,EventOrigin,OfferID,lifecycle:transition,dt,OfferID_was_null
0,0,Offer_247135719,O_Create Offer,User_17,Variant 8,8,True,Application_196483749,0,10000.0,...,57,10000.0,False,Created,Offer_247135719,Offer,Offer_247135719,complete,0 days 00:00:03.042000,True
1,1,Offer_247135719,O_Created,User_17,Variant 8,8,True,Application_196483749,0,10000.0,...,57,10000.0,False,statechange,OfferState_124849367,Offer,Offer_247135719,complete,0 days 00:02:12.568000,False
2,2,Offer_247135719,O_Sent (online only),User_17,Variant 8,8,True,Application_196483749,0,10000.0,...,57,10000.0,False,statechange,OfferState_440662877,Offer,Offer_247135719,complete,0 days 00:02:04.704000,False
3,3,Offer_247135719,O_Cancelled,User_17,Variant 8,8,True,Application_196483749,0,10000.0,...,57,10000.0,False,statechange,OfferState_591416028,Offer,Offer_247135719,complete,0 days 00:02:04.704000,False
4,4,Offer_941964966,O_Create Offer,User_17,Variant 1,1,True,Application_196483749,0,4100.0,...,57,10000.0,False,Created,Offer_941964966,Offer,Offer_941964966,complete,0 days 00:00:01.551000,True


In [10]:
df_grouped = df.groupby(["caseid"]).agg(list).reset_index()

In [11]:
df_grouped = Utils.reduce_list_columns(df_grouped)

In [12]:
Utils.display_columns_to_aggragate(df_grouped)

,activity,resource,Action,EventID,OfferID,dt,OfferID_was_null
0,"[O_Create Offer, O_Created, O_Sent (mail and o...","[User_2, User_2, User_2, User_113, User_30]","[Created, statechange, statechange, statechang...","[Offer_1001553250, OfferState_1604351174, Offe...","[Offer_533969936, Offer_1001553250, Offer_1001...","[0 days 00:00:01.188000, 0 days 00:00:15.47200...","[True, False, False, False, False]"
1,"[O_Create Offer, O_Created, O_Sent (mail and o...","[User_85, User_85, User_85, User_117, User_118]","[Created, statechange, statechange, statechang...","[Offer_1002136393, OfferState_1608457034, Offe...","[Offer_935859805, Offer_1002136393, Offer_1002...","[0 days 00:00:01.293000, 0 days 00:00:19.55600...","[True, False, False, False, False]"
2,"[O_Create Offer, O_Created, O_Sent (mail and o...","[User_49, User_49, User_49, User_113, User_102]","[Created, statechange, statechange, statechang...","[Offer_1002236598, OfferState_1780384890, Offe...","[Offer_1651128996, Offer_1002236598, Offer_100...","[0 days 00:00:01.269000, 0 days 00:00:11.34800...","[True, False, False, False, False]"
3,"[O_Create Offer, O_Created, O_Sent (mail and o...","[User_18, User_18, User_18, User_115]","[Created, statechange, statechange, statechange]","[Offer_1003919273, OfferState_1789429921, Offe...","[Offer_1638445832, Offer_1003919273, Offer_100...","[0 days 00:00:01.256000, 0 days 00:00:20.98200...","[True, False, False, False]"
4,"[O_Create Offer, O_Created, O_Sent (mail and o...","[User_2, User_2, User_2, User_1]","[Created, statechange, statechange, statechange]","[Offer_1004444125, OfferState_1945050329, Offe...","[Offer_520979602, Offer_1004444125, Offer_1004...","[0 days 00:00:01.255000, 0 days 00:00:21.83300...","[True, False, False, False]"
...,...,...,...,...,...,...,...
2495,"[O_Create Offer, O_Created, O_Sent (mail and o...","[User_5, User_5, User_5, User_112, User_68]","[Created, statechange, statechange, statechang...","[Offer_990346100, OfferState_1443056976, Offer...","[Offer_464709031, Offer_990346100, Offer_99034...","[0 days 00:00:01.278000, 0 days 00:00:16.93500...","[True, False, False, False, False]"
2496,"[O_Create Offer, O_Created, O_Sent (online onl...","[User_28, User_28, User_28, User_118, User_118]","[Created, statechange, statechange, statechang...","[Offer_990618639, OfferState_1139979457, Offer...","[Offer_520979602, Offer_990618639, Offer_99061...","[0 days 00:00:01.620000, 0 days 00:00:30.18800...","[True, False, False, False, False]"
2497,"[O_Create Offer, O_Created, O_Sent (mail and o...","[User_25, User_25, User_25, User_34]","[Created, statechange, statechange, statechange]","[Offer_993451947, OfferState_991935059, OfferS...","[Offer_520979602, Offer_993451947, Offer_99345...","[0 days 00:00:01.313000, 0 days 00:00:19.59500...","[True, False, False, False]"
2498,"[O_Create Offer, O_Created, O_Sent (mail and o...","[User_27, User_27, User_27, User_119]","[Created, statechange, statechange, statechange]","[Offer_993469235, OfferState_280089061, OfferS...","[Offer_1468908520, Offer_993469235, Offer_9934...","[0 days 00:00:01.201000, 0 days 00:01:00.10000...","[True, False, False, False]"


In [13]:
#Some colomns have no importance in the process so we drop them
df_grouped = df_grouped.drop(['EventID','OfferID'],axis=1)
df_grouped = df_grouped.drop(['OfferID_was_null'],axis=1)

In [14]:
df_grouped.head(2)

,caseid,activity,resource,Variant,Variant index,y,(case) ApplicationID,(case) CreditScore,(case) FirstWithdrawalAmount,(case) MonthlyCost,(case) NumberOfTerms,(case) OfferedAmount,(case) Selected,Action,EventOrigin,lifecycle:transition,dt
0,Offer_1001553250,"[O_Create Offer, O_Created, O_Sent (mail and o...","[User_2, User_2, User_2, User_113, User_30]",Variant 2,2,False,Application_1607028451,0,2500.0,175.00,127,17500.0,True,"[Created, statechange, statechange, statechang...",Offer,complete,"[0 days 00:00:01.188000, 0 days 00:00:15.47200..."
1,Offer_1002136393,"[O_Create Offer, O_Created, O_Sent (mail and o...","[User_85, User_85, User_85, User_117, User_118]",Variant 2,2,True,Application_638926349,969,6500.0,343.25,20,6500.0,True,"[Created, statechange, statechange, statechang...",Offer,complete,"[0 days 00:00:01.293000, 0 days 00:00:19.55600..."


In [15]:
df_grouped_agg = Utils.aggregation_encoding(df_grouped,df)

In [16]:
df_grouped_agg.drop("dt",axis=1,inplace = True)

In [17]:
df_grouped_agg.head()

,caseid,Variant,Variant index,y,(case) ApplicationID,(case) CreditScore,(case) FirstWithdrawalAmount,(case) MonthlyCost,(case) NumberOfTerms,(case) OfferedAmount,...,User_23,User_73,User_125,User_86,User_142,Created,statechange,avg_dt,max_dt,min_dt
0,Offer_1001553250,Variant 2,2,False,Application_1607028451,0,2500.0,175.00,127,17500.0,...,0.0,0.0,0.0,0.0,0.0,0.20,0.80,2 days 22:47:28.444400,6 days 13:57:17.486000,0 days 00:00:01.188000
1,Offer_1002136393,Variant 2,2,True,Application_638926349,969,6500.0,343.25,20,6500.0,...,0.0,0.0,0.0,0.0,0.0,0.20,0.80,5 days 22:58:29.630800,18 days 00:58:56.495000,0 days 00:00:01.293000
2,Offer_1002236598,Variant 3,3,False,Application_818693455,0,2500.0,210.00,127,21000.0,...,0.0,0.0,0.0,0.0,0.0,0.20,0.80,1 days 20:42:58.900800,3 days 03:26:35.948000,0 days 00:00:01.269000
3,Offer_1003919273,Variant 6,6,False,Application_941143913,0,0.0,160.44,83,10900.0,...,0.0,0.0,0.0,0.0,0.0,0.25,0.75,8 days 12:12:41.137000,17 days 00:25:11.155000,0 days 00:00:01.256000
4,Offer_1004444125,Variant 1,1,True,Application_1498056352,0,11500.0,202.07,67,11500.0,...,0.0,0.0,0.0,0.0,0.0,0.25,0.75,25 days 10:16:47.452500,50 days 20:33:23.361000,0 days 00:00:01.255000


## Logs encoding

i think: activity - onehot?, variant - one hot, action, onehot, event origin - onehot, liofecycle - one hot

In [18]:
import h2o
from h2o.automl import H2OAutoML

In [19]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,14 mins 43 secs
H2O_cluster_timezone:,Europe/Berlin
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.3
H2O_cluster_version_age:,27 days
H2O_cluster_name:,H2O_from_python_l_marazzi_rjo86d
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.711 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [20]:
X_train, X_test = train_test_split(df_grouped_agg, test_size=0.3)

df_h_train = h2o.H2OFrame(X_train)
df_h_test = h2o.H2OFrame(X_test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [21]:
x = df_h_train.columns
y = "y"
x.remove(y)

In [22]:
df_h_train[y] = df_h_train[y].asfactor()
df_h_train[y] = df_h_train[y].asfactor()

In [ ]:
aml = H2OAutoML(max_models=20, seed=1)
aml.train(x=x, y=y, training_frame=df_h_train)

AutoML progress: |█
01:36:27.38: AutoML: XGBoost is not available; skipping it.
01:36:27.38: _train param, Dropping bad and constant columns: [max_dt, caseid, User_33, EventOrigin, avg_dt, lifecycle:transition, User_125]
01:36:29.924: _train param, Dropping bad and constant columns: [max_dt, caseid, User_33, EventOrigin, avg_dt, lifecycle:transition, User_125]
01:36:31.320: _train param, Dropping bad and constant columns: [max_dt, caseid, User_33, EventOrigin, avg_dt, lifecycle:transition, User_125]

██
01:36:33.192: _train param, Dropping bad and constant columns: [max_dt, caseid, User_33, EventOrigin, avg_dt, lifecycle:transition, User_125]
01:36:34.526: _train param, Dropping bad and constant columns: [max_dt, caseid, User_33, EventOrigin, avg_dt, lifecycle:transition, User_125]
01:36:36.116: _train param, Dropping bad and constant columns: [max_dt, caseid, User_33, EventOrigin, avg_dt, lifecycle:transition, User_125]

███
01:36:37.806: _train param, Dropping bad and constant column

In [ ]:
lb = aml.leaderboard
lb.head(rows=lb.nrows) 

In [ ]:
model = aml.leader

In [ ]:
perf = model.model_performance(df_h_test)
perf

## Model selection

## Evaluation of results